In [3]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
gpt_turbo = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [33]:
import requests
url = 'https://public-api.meteofrance.fr/public/DPVigilance/v1/textesvigilance/encours'
headers = {
    'accept': '*/*',
    'apikey': os.environ['VIGILENCE_API_KEY']
}
response = requests.get(url, headers=headers)
data = response.json()

In [34]:
def extract_text_items(data):
    text_items = []
    infos = [
        region for region in data['product']['text_bloc_items'] 
        if len(region['bloc_items']) != 0 and region['bloc_id']=='BULLETIN_DEPARTEMENTAL'
    ]
    for info in infos:
        for bloc_item in info.get('bloc_items', []):
            for text_item in bloc_item.get('text_items', []):
                for subdivision in text_item.get('term_items', []):
                    for subdivision_text in subdivision.get('subdivision_text', []):
                        text_items.append("".join(subdivision_text.get('text', '')))
    return text_items

In [35]:
infos = list(set(extract_text_items(data)))

In [36]:
template = """
Voici les informations concernant les vigilences du jour peux tu me faire un résumé en français : {informations}
"""

In [37]:
prompt = PromptTemplate.from_template(template)


In [38]:
", ".join(infos)

'Faits nouveaux : passage en vigilance orange vent violents sur le Finistère pour un début d\'évènement à 16h ce lundi. Situation générale : Une dépression nommée PIERRICK se présente en entrée de Manche, proche du nord Finistère en ce début d\'après-midi de lundi puis circule le long des côtes sud de l\'Angleterre, engendrant des vents violents et un risque de vagues-submersion., La dépression PIERRICK concerne dans un premier temps le Finistère cet après-midi jusqu\'en soirée. La dépression se décale ensuite vers l\'est et donne encore de fortes rafales sur le département des Côtes d\'Armor puis concerne le département de la Manche la nuit suivante. On attend 120 à 130 km/h sur les îles et à la côte du Finistère, autour de 110 km/h proche des littoraux et sur un quart nord-ouest du Finistère, 90 à ponctuellement plus de 100 km/h ailleurs sur le département. , Une incertitude persiste sur le positionnement exacte du creusement lié à PIERRICK. Ainsi on privilégie un scénario avec de vi

In [39]:
chain = {'informations':RunnablePassthrough() } | prompt | gpt_turbo

In [40]:
chain.invoke(infos)

'\nAujourd\'hui, la vigilance concerne principalement le Finistère, où des vents violents sont attendus avec l\'arrivée de la dépression PIERRICK. Celle-ci devrait ensuite se déplacer vers l\'est, touchant également les départements des Côtes d\'Armor et de la Manche avec des rafales pouvant atteindre jusqu\'à 130 km/h. Une incertitude persiste quant au positionnement exact de la dépression, mais il est prévu qu\'elle provoque également des vents forts dans les terres et sur la côte sud du Finistère. Une alerte pour les risques de vagues-submersion a également été émise pour les départements de l\'est de la Manche. Demain, la dépression se déplacera vers l\'Angleterre et la Mer du Nord, générant de fortes vagues sur la Manche et la côte atlantique. Les départements placés en vigilance "vagues-submersion" de niveau orange sont ceux du Finistère, des Côtes d\'Armor, de l\'Ille-et-Vilaine et de l\'ouest de la Manche'